# ANALYSING AND CLEANING PUMPKIN DATA

In [6]:
import pandas as pd

# Read the data from the CSV file
pumpkins_raw_data = pd.read_csv('../data/US-pumpkins.csv')

# Display the first 5 rows of the data

print(f"The first 5 rows of the data are: \n{pumpkins_raw_data.head()}")



The first 5 rows of the data are: 
   City Name Type       Package      Variety Sub Variety  Grade     Date  \
0  BALTIMORE  NaN  24 inch bins          NaN         NaN    NaN  4/29/17   
1  BALTIMORE  NaN  24 inch bins          NaN         NaN    NaN   5/6/17   
2  BALTIMORE  NaN  24 inch bins  HOWDEN TYPE         NaN    NaN  9/24/16   
3  BALTIMORE  NaN  24 inch bins  HOWDEN TYPE         NaN    NaN  9/24/16   
4  BALTIMORE  NaN  24 inch bins  HOWDEN TYPE         NaN    NaN  11/5/16   

   Low Price  High Price  Mostly Low  ...  Unit of Sale Quality Condition  \
0      270.0       280.0       270.0  ...           NaN     NaN       NaN   
1      270.0       280.0       270.0  ...           NaN     NaN       NaN   
2      160.0       160.0       160.0  ...           NaN     NaN       NaN   
3      160.0       160.0       160.0  ...           NaN     NaN       NaN   
4       90.0       100.0        90.0  ...           NaN     NaN       NaN   

  Appearance Storage  Crop Repack  Trans Mode

## Below we Filter out any rows that don't use price per bushel in the package column.

In [9]:
# Filter out any rows that don't use price per bushel
pumpkins_bushel_data = pumpkins_raw_data[pumpkins_raw_data["Package"].str.contains("bushel", case=True, regex=True)] # The case=False argument is used to ignore the case of the string and the regex=True argument is used to enable the use of regular expressions in the filter method of the pandas library like the contains method used in this case to filter out the rows that contain the string "bushel" in the "Package" column of the data 

# Display the first 5 rows of the filtered data
print(f"The first 5 rows of the filtered data are: \n{pumpkins_bushel_data.head()}")


The first 5 rows of the filtered data are: 
    City Name Type               Package   Variety Sub Variety  Grade  \
70  BALTIMORE  NaN  1 1/9 bushel cartons  PIE TYPE         NaN    NaN   
71  BALTIMORE  NaN  1 1/9 bushel cartons  PIE TYPE         NaN    NaN   
72  BALTIMORE  NaN  1 1/9 bushel cartons  PIE TYPE         NaN    NaN   
73  BALTIMORE  NaN  1 1/9 bushel cartons  PIE TYPE         NaN    NaN   
74  BALTIMORE  NaN  1 1/9 bushel cartons  PIE TYPE         NaN    NaN   

       Date  Low Price  High Price  Mostly Low  ...  Unit of Sale Quality  \
70  9/24/16       15.0        15.0        15.0  ...           NaN     NaN   
71  9/24/16       18.0        18.0        18.0  ...           NaN     NaN   
72  10/1/16       18.0        18.0        18.0  ...           NaN     NaN   
73  10/1/16       17.0        17.0        17.0  ...           NaN     NaN   
74  10/8/16       15.0        15.0        15.0  ...           NaN     NaN   

   Condition Appearance Storage  Crop Repack  Trans Mo

## Count how many cellls are empty in each column

In [10]:
# Count how many cells are emptty in each column
pumpkins_bushel_data.isnull().sum() # The isnull() method is used to check for missing values in the data and the sum() method is used to count the number of missing values in each column of the data

City Name            0
Type               406
Package              0
Variety              0
Sub Variety        167
Grade              415
Date                 0
Low Price            0
High Price           0
Mostly Low          24
Mostly High         24
Origin               0
Origin District    396
Item Size          114
Color              145
Environment        415
Unit of Sale       404
Quality            415
Condition          415
Appearance         415
Storage            415
Crop               415
Repack               0
Trans Mode         415
Unnamed: 24        415
Unnamed: 25        391
dtype: int64

## Since the columns we are interested in are not empty, we proceed to get the columns we want to work with. Which are the package, date,low price and high price columns.

In [13]:
#  Define the columns that we want to keep
new_columns = ["Package", "Month", "Low Price", "High Price", "Date"]

# Create a new DataFrame with only the columns we want
pumpkins_selected_column_data = pumpkins_bushel_data.drop([c for c in pumpkins_bushel_data.columns if c not in new_columns], axis=1) # The drop() method is used to remove the columns that are not in the new_columns list from the data the axis=1 argument is used to specify that the columns are to be removed and not the rows if the axis=0 argument is used the rows will be removed

# Display the first 5 rows of the new DataFrame
print(f"The first 5 rows of the new DataFrame are: \n{pumpkins_selected_column_data.head()}")

The first 5 rows of the new DataFrame are: 
                 Package     Date  Low Price  High Price
70  1 1/9 bushel cartons  9/24/16       15.0        15.0
71  1 1/9 bushel cartons  9/24/16       18.0        18.0
72  1 1/9 bushel cartons  10/1/16       18.0        18.0
73  1 1/9 bushel cartons  10/1/16       17.0        17.0
74  1 1/9 bushel cartons  10/8/16       15.0        15.0


## Calculate the average price of pumpkins and extract the month from the date column.

In [14]:
# Calculate the average price of pumpkins from the low and high prices
price = (pumpkins_selected_column_data["Low Price"] + pumpkins_selected_column_data["High Price"]) / 2

# Get the month from the date column
month = pd.DatetimeIndex(pumpkins_selected_column_data["Date"]).month

# Create a new DataFrame with the following columns
new_pumpkins_data = pd.DataFrame({"Package": pumpkins_selected_column_data["Package"], "Month": month, "Low Price": pumpkins_selected_column_data["Low Price"], "High Price": pumpkins_selected_column_data["High Price"], "Avg Price": price})

# Display the first 5 rows of the new DataFrame
print(f"The first 5 rows of the new DataFrame are: \n{new_pumpkins_data.head()}")

The first 5 rows of the new DataFrame are: 
                 Package  Month  Low Price  High Price  Avg Price
70  1 1/9 bushel cartons      9       15.0        15.0       15.0
71  1 1/9 bushel cartons      9       18.0        18.0       18.0
72  1 1/9 bushel cartons     10       18.0        18.0       18.0
73  1 1/9 bushel cartons     10       17.0        17.0       17.0
74  1 1/9 bushel cartons     10       15.0        15.0       15.0


## Calculate price per bushel

In [19]:
# Convert the price of all cell price by 1 1/9 bushels by dividing by 1 1/9
new_pumpkins_data.loc[new_pumpkins_data["Package"].str.contains("1 1/9"), "Avg Price"] = price/(1 + 1/9) # The loc[] method is used to access a group of rows and columns by labels or a boolean array.

# Convert the price of all cell price by 1/2 bushels by dividing by 1/2
new_pumpkins_data.loc[new_pumpkins_data["Package"].str.contains("1/2"), "Avg Price"] = price/(1/2)

# Display the first 5 rows of the new DataFrame
print(f"The first 10 rows of the new DataFrame are: \n{new_pumpkins_data.head()}")

The first 10 rows of the new DataFrame are: 
                 Package  Month  Low Price  High Price  Avg Price
70  1 1/9 bushel cartons      9       15.0        15.0       13.5
71  1 1/9 bushel cartons      9       18.0        18.0       16.2
72  1 1/9 bushel cartons     10       18.0        18.0       16.2
73  1 1/9 bushel cartons     10       17.0        17.0       15.3
74  1 1/9 bushel cartons     10       15.0        15.0       13.5
